<a href="https://colab.research.google.com/github/Santosdevbjj/relatoVendasLucros/blob/main/notebooks/exploracao_estatistica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# 🧮 Exploração Estatística — Relatório de Vendas e Lucros\n",
    "Este notebook realiza uma análise exploratória e estatística detalhada dos dados contidos nos arquivos `vendas.csv`, `produtos.csv`, `clientes.csv` e `regioes.csv`.\n",
    "\n",
    "O objetivo é compreender padrões, identificar correlações e gerar insights que poderiam ser traduzidos visualmente em um dashboard Power BI — mesmo sem utilizá-lo diretamente."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Importação das bibliotecas essenciais\n",
    "import pandas as pd\n",
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "import seaborn as sns\n",
    "from scipy import stats\n",
    "plt.style.use('ggplot')"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1️⃣ Carregamento e visualização inicial dos dados"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 2,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Carrega os arquivos CSV\n",
    "vendas = pd.read_csv('../data/vendas.csv')\n",
    "produtos = pd.read_csv('../data/produtos.csv')\n",
    "clientes = pd.read_csv('../data/clientes.csv')\n",
    "regioes = pd.read_csv('../data/regioes.csv')"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 3,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Exibe uma amostra dos dados de vendas\n",
    "vendas.head()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2️⃣ Verificação e limpeza básica dos dados"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 4,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Informações gerais sobre o dataset\n",
    "vendas.info()\n",
    "\n",
    "# Verifica valores nulos\n",
    "vendas.isna().sum()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 5,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Remove registros duplicados, se houver\n",
    "vendas.drop_duplicates(inplace=True)\n",
    "\n",
    "# Corrige tipos de dados\n",
    "vendas['data_venda'] = pd.to_datetime(vendas['data_venda'])\n",
    "vendas['lucro'] = vendas['lucro'].astype(float)\n",
    "vendas['preco_venda'] = vendas['preco_venda'].astype(float)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3️⃣ Estatísticas descritivas e visão geral"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 6,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Estatísticas básicas\n",
    "vendas.describe()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4️⃣ Análise de vendas e lucros por categoria de produto"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 7,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Junta os datasets\n",
    "dados_completos = vendas.merge(produtos, on='id_produto').merge(clientes, on='id_cliente').merge(regioes, on='id_regiao')\n",
    "\n",
    "# Agrupa por categoria de produto\n",
    "resumo_categoria = dados_completos.groupby('categoria_produto')[['quantidade', 'lucro']].sum().reset_index()\n",
    "resumo_categoria"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 8,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Gráfico de barras: Lucro por categoria de produto\n",
    "plt.figure(figsize=(8,5))\n",
    "sns.barplot(data=resumo_categoria, x='categoria_produto', y='lucro', palette='coolwarm')\n",
    "plt.title('Lucro Total por Categoria de Produto')\n",
    "plt.xlabel('Categoria')\n",
    "plt.ylabel('Lucro (R$)')\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 5️⃣ Análise temporal — Tendência mensal de vendas e lucros"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 9,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Criação de colunas auxiliares\n",
    "dados_completos['ano_mes'] = dados_completos['data_venda'].dt.to_period('M')\n",
    "\n",
    "# Agrupamento mensal\n",
    "tendencia = dados_completos.groupby('ano_mes')[['quantidade', 'lucro']].sum().reset_index()\n",
    "\n",
    "# Gráfico de linha\n",
    "plt.figure(figsize=(10,5))\n",
    "plt.plot(tendencia['ano_mes'].astype(str), tendencia['lucro'], marker='o')\n",
    "plt.title('Tendência Mensal de Lucros')\n",
    "plt.xticks(rotation=45)\n",
    "plt.ylabel('Lucro (R$)')\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 6️⃣ Correlação entre Preço, Quantidade e Lucro"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 10,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Matriz de correlação\n",
    "corr = dados_completos[['preco_venda', 'quantidade', 'lucro']].corr()\n",
    "\n",
    "# Heatmap\n",
    "plt.figure(figsize=(6,4))\n",
    "sns.heatmap(corr, annot=True, cmap='Blues', fmt='.2f')\n",
    "plt.title('Correlação entre Preço, Quantidade e Lucro')\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 7️⃣ Análise de Lucro por Região"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 11,
   "metadata": {},
   "outputs": [],
   "source": [
    "resumo_regiao = dados_completos.groupby('nome_regiao')[['lucro']].sum().sort_values(by='lucro', ascending=False)\n",
    "plt.figure(figsize=(8,5))\n",
    "sns.barplot(x=resumo_regiao.index, y=resumo_regiao['lucro'], palette='viridis')\n",
    "plt.title('Lucro Total por Região')\n",
    "plt.ylabel('Lucro (R$)')\n",
    "plt.xlabel('Região')\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 8️⃣ Teste estatístico — Distribuição de lucros\n",
    "Vamos verificar se os lucros seguem uma distribuição normal (usando Shapiro-Wilk)."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 12,
   "metadata": {},
   "outputs": [],
   "source": [
    "stat, p = stats.shapiro(dados_completos['lucro'])\n",
    "print(f'Estatística do teste: {stat:.3f} | Valor-p: {p:.5f}')\n",
    "\n",
    "if p > 0.05:\n",
    "    print('Os lucros seguem uma distribuição aproximadamente normal.')\n",
    "else:\n",
    "    print('Os lucros NÃO seguem uma distribuição normal.')"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 9️⃣ Conclusões gerais\n",
    "\n",
    "✅ Categorias de produtos com maiores lucros: observadas nas barras superiores do gráfico.\n",
    "\n",
    "✅ Regiões mais lucrativas: destaque para aquelas com maiores barras no gráfico de região.\n",
    "\n",
    "✅ Correlação positiva entre preço de venda e lucro — porém não tão forte com quantidade.\n",
    "\n",
    "✅ Lucros não seguem distribuição normal, sugerindo concentração em poucos produtos ou clientes.\n",
    "\n",
    "---\n",
    "**📈 Insight final:** esses resultados formam a base para o dashboard Power BI, que pode usar as mesmas agregações e filtros mostrados aqui (em pandas + matplotlib)."
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "name": "python",
   "version": "3.11"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 2
}